In [ ]:
import sys
import os

# Go one level up from "notebook/" to the root
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [ ]:
# Now this will work:
from config.settings import load_config, get_groq_headers
# Load configuration settings
config = load_config()
print("✅ Config Loaded:")
print(config)

In [ ]:
def get_llm_response(
    prompt: str,
    system_message: str = "You are a helpful assistant.",
    model: str = None,
    max_tokens: int = None,
    temperature: float = None
) -> str:
    """
    Get response from Groq LLM API using Llama3-8B-8192 model
    
    Args:
        prompt: User prompt/question
        system_message: System context message
        model: Model to use (defaults to config)
        max_tokens: Max tokens to generate
        temperature: Temperature for generation
    
    Returns:
        Generated text response
    """
    
    # Use config defaults if not specified
    if model is None:
        model = config["groq_model"]
    if max_tokens is None:
        max_tokens = config["max_tokens"]
    if temperature is None:
        temperature = config["temperature"]
    
    # Prepare request payload
    payload = {
        "model": model,
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt}
        ],
        "max_tokens": max_tokens,
        "temperature": temperature,
        "top_p": 1,
        "stream": False
    }
    
    # Get headers
    headers = get_groq_headers(config["groq_api_key"])
    
    # Make API request with retries
    for attempt in range(config["max_retries"]):
        try:
            response = requests.post(
                config["groq_api_url"],
                headers=headers,
                json=payload,
                timeout=30
            )
            
            if response.status_code == 200:
                response_data = response.json()
                
                # Extract generated text
                if "choices" in response_data and len(response_data["choices"]) > 0:
                    content = response_data["choices"][0]["message"]["content"]
                    return content.strip()
                else:
                    raise Exception("No choices in response")
                    
            elif response.status_code == 429:  # Rate limit
                wait_time = 2 ** attempt
                time.sleep(wait_time)
                continue
            else:
                raise Exception(f"API request failed with status {response.status_code}: {response.text}")
                
        except requests.exceptions.RequestException as e:
            if attempt == config["max_retries"] - 1:
                raise Exception(f"Failed to get LLM response after {config['max_retries']} attempts: {str(e)}")
            
            # Wait before retry
            wait_time = 2 ** attempt
            time.sleep(wait_time)
    
    raise Exception("Failed to get LLM response")


prompt = "Write a short motivational quote."
response = get_llm_response(prompt)
print("📜 LLM Response:\n", response)


In [ ]:
from typing import Dict, Any, Optional
def get_llm_response_with_context(
    prompt: str,
    context: Dict[str, Any],
    system_message: str = "You are a helpful assistant."
) -> str:
    """
    Get LLM response with additional context information
    
    Args:
        prompt: User prompt/question
        context: Additional context information
        system_message: System context message
    
    Returns:
        Generated text response
    """
    
    # Build enhanced prompt with context
    context_str = format_context_for_prompt(context)
    enhanced_prompt = f"{context_str}\n\n{prompt}"
    
    return get_llm_response(enhanced_prompt, system_message)


context = {
    "product_name": "Green Matcha Energy Drink",
    "target_audience": ["Gen Z"]
}
prompt = "Write a product tagline."
response = get_llm_response_with_context(prompt, context)
print("🧠 Contextual LLM Response:\n", response)

In [ ]:
def format_context_for_prompt(context: Dict[str, Any]) -> str:
    """Format context dictionary for inclusion in prompt"""
    
    context_lines = ["Context Information:"]
    
    for key, value in context.items():
        if isinstance(value, (str, int, float)):
            context_lines.append(f"- {key.replace('_', ' ').title()}: {value}")
        elif isinstance(value, list):
            if value:  # Only add if list is not empty
                context_lines.append(f"- {key.replace('_', ' ').title()}: {', '.join(map(str, value))}")
        elif isinstance(value, dict):
            context_lines.append(f"- {key.replace('_', ' ').title()}:")
            for sub_key, sub_value in value.items():
                context_lines.append(f"  - {sub_key.replace('_', ' ').title()}: {sub_value}")
    
    return '\n'.join(context_lines)

In [ ]:
from typing import Dict, Any, Optional
def get_huggingface_headers(api_key: str) -> Dict[str, str]:
    """Get headers for HuggingFace API requests"""
    return {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
get_huggingface_headers('dfsdfsdfsdfs')